In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from google.colab import drive

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Tentukan Path untuk Input dan Output
input_path = "/content/drive/MyDrive/Dataset_Telur_Skripsi"
output_path = "/content/drive/MyDrive/Result_Preprocessed(fix)"

# 3. Folder kategori
classes = ['Berkualitas', 'Kurang Berkualitas', 'Tidak Berkualitas']

# 4. Memastikan folder output ada
if not os.path.exists(output_path):
    os.makedirs(output_path)

Mounted at /content/drive


In [ ]:
# 5. Fungsi untuk Crop Gambar Fokus pada Telur Menggunakan Deteksi Kontur
def crop_to_egg(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)  # Gunakan kernel yang lebih besar
    _, thresholded = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    else:
        return image

# 6. Fungsi untuk Menerapkan Gaussian Blur pada Gambar (Kernel Kecil)
def apply_gaussian_blur(image, kernel_size=(5, 5)):
    blurred_image = cv2.GaussianBlur(image, kernel_size, 0)
    return blurred_image

# 7. Fungsi untuk Menerapkan CLAHE (Contrast Limited Adaptive Histogram Equalization) dengan clipLimit
def apply_clahe(image, clipLimit=2.0, tileGridSize=(8, 8)):  # Cobalah dengan clipLimit yang lebih moderat
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    enhanced_image = clahe.apply(image)
    return enhanced_image

# 8. Fungsi untuk Resize dan Padding Gambar
def resize_and_pad(image, desired_size=(256, 256)):
    old_size = image.shape[:2]
    ratio = float(desired_size[0]) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    resized_img = cv2.resize(image, (new_size[1], new_size[0]))

    delta_w = desired_size[1] - new_size[1]
    delta_h = desired_size[0] - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    padded_img = cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    return padded_img

# 9. Fungsi untuk Augmentasi (Flip dan Rotasi)
def augment_image_safe_versions(image):
    flip = cv2.flip(image, 1)  # Flip horizontal
    rotate90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    return [flip, rotate90]

# 10. Fungsi untuk Preprocessing Semua Gambar + Augmentasi Aman
def process_preprocessing(input_path, output_path):
    for kls in classes:
        input_dir = os.path.join(input_path, kls)
        output_dir = os.path.join(output_path, kls)
        os.makedirs(output_dir, exist_ok=True)

        for filename in tqdm(os.listdir(input_dir), desc=f"Preprocessing {kls}"):
            if filename.lower().endswith(".jpg"):
                path_img = os.path.join(input_dir, filename)
                img = cv2.imread(path_img)

                # 1. Crop gambar berdasarkan kontur telur
                cropped_image = crop_to_egg(img)

                # 2. Ubah gambar menjadi grayscale
                gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

                # 3. Preprocessing: blur, CLAHE
                blurred_image = apply_gaussian_blur(gray_image, kernel_size=(7, 7))  # Gunakan kernel yang lebih besar
                clahe_image = apply_clahe(blurred_image, clipLimit=2.0)  # Menurunkan clipLimit agar lebih natural
                final_image = resize_and_pad(clahe_image)

                # 4. Simpan gambar hasil preprocessing
                base_name = os.path.splitext(filename)[0]
                save_path = os.path.join(output_dir, f"{base_name}_preprocessed.jpg")
                cv2.imwrite(save_path, final_image)

                # 5. Lakukan augmentasi
                augmentations = augment_image_safe_versions(final_image)
                for idx, aug_img in enumerate(augmentations):
                    aug_save_path = os.path.join(output_dir, f"{base_name}_aug{idx+1}.jpg")
                    cv2.imwrite(aug_save_path, aug_img)

In [ ]:
# 7. Panggil proses preprocessing
process_preprocessing(input_path, output_path)

Preprocessing Tidak Berkualitas: 100%|██████████| 270/270 [01:32<00:00,  2.90it/s]
